In [5]:
import numpy as np
import pandas as pd
import os
import copy
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

pd.set_option('display.max_columns', None)


In [6]:
testPath = '../data/eleme_round1_testB/' # B榜数据
#testPath = '../data/eleme_round1_testA/' # A榜数据
data_list= [str(i) for i in range(20200301,20200307)]
def getDF(path,fileName):
    df_list =[]
    for f in data_list:
        fn = fileName+'_'+f+'.txt'
        df = pd.read_csv(os.path.join(path,fileName,fn))
        df_list.append(df)
       
    return df_list

In [7]:
# 数据读取
test_action_list= getDF(testPath,'action')
test_courier_list = getDF(testPath,'courier')
test_distance_list = getDF(testPath,'distance')
test_order_list= getDF(testPath,'order')

In [8]:
# 列名字典
action_columns_name_dict={'courier_id':'骑手ID','wave_index':'波次','tracking_id':'订单号','courier_wave_start_lng':'骑手起始经度','courier_wave_start_lat':'骑手起始纬度','action_type':'动作类型','expect_time':'时间'}
courier_columns_name_dict={'courier_id':'骑手ID','level':'等级','speed':'速度','max_load':'最大负荷量'}
distance_columns_name_dict={'courier_id':'骑手ID','wave_index':'波次','tracking_id':'源订单号','source_type':'源类型','source_lng':'源经度','source_lat':'源纬度','target_tracking_id':'目标订单号','target_type':'目标类型','target_lng':'目标经度','target_lat':'目标纬度','grid_distance':'距离'}
order_columns_name_dict={'courier_id':'骑手ID','wave_index':'波次','tracking_id':'订单号','weather_grade':'天气','pick_lng':'商家纬度','pick_lat':'商家经度','deliver_lng':'客户纬度','deliver_lat':'客户经度','create_time':'创建时间','confirm_time':'确认时间','assigned_time':'通知时间','promise_deliver_time':'承诺送达时间','estimate_pick_time':'可以取单时间','aoi_id':'客户标识ID','shop_id':'商家标识ID'}

In [9]:
 # 获取每个订单动作后续取单和送单对应的可选单
def OneOrderActionGetAllOption(OrderID,Action,ActionDF,OrderDF,DistanceDF,shuxing):
    actiondf = ActionDF.loc[ActionDF['时间'] !=0,:]
    actiondf.sort_values(by='时间',inplace = True)
    action_time = actiondf.loc[(actiondf['订单号']==OrderID) & (actiondf['动作类型']==Action),'时间'].values[0]
    order_timeBefor = actiondf.loc[actiondf['时间'] <= action_time,:] # 小于当前动作完成前的所有订单及对应动作
    order_assign = pd.DataFrame(columns=['订单号','动作类型','时间','经度','纬度']) # 存放已发布，未提取
    order_pickup = pd.DataFrame(columns=['订单号','动作类型','时间','经度','纬度']) # 存放已取单，未送单
    last_action = order_timeBefor.iloc[-1,:] # 决策前的最后一个动作
    # 挑出
    for j in range(len(order_timeBefor)):
        orderid = order_timeBefor.iloc[j,0] 
        orderDF = order_timeBefor.loc[order_timeBefor['订单号']== orderid,:]
        if len(orderDF) == 1:
            order_assign=order_assign.append(ActionDF.loc[(ActionDF['订单号']==orderDF.iloc[0,0]) & (ActionDF['动作类型'] =='PICKUP'),:],ignore_index =True)
        elif len(orderDF) == 2:
            df1 = orderDF.iloc[1,:]
            order_pickup=order_pickup.append(ActionDF.loc[(ActionDF['订单号']==df1.iloc[0]) & (ActionDF['动作类型'] =='DELIVERY'),:],ignore_index =True)
            #order_timeBefor.loc[order_timeBefor['订单号']==orderid,:] =0
            #print(order_timeBefor)
            #print(last_action)
    # 去除重复
    order_pickup.drop_duplicates(inplace=True)
    
    # 距离
    order_assign['距离'] =0 # 骑手距离商家
    order_pickup['距离'] =0 # 骑手距离客户
  
    for y in range(len(order_assign)):
        order_assign.iloc[y,5] = DistanceDF.loc[(DistanceDF['源订单号']==last_action['订单号']) & (DistanceDF['源类型']==last_action['动作类型'])&(DistanceDF['目标订单号']==order_assign.iloc[y,0])&(DistanceDF['目标类型']==order_assign.iloc[y,1]),'距离'].values[0]
    for z in range(len(order_pickup)):
        try:
            order_pickup.iloc[z,5] = DistanceDF.loc[(DistanceDF['源订单号']==last_action['订单号']) & (DistanceDF['源类型']==last_action['动作类型'])&(DistanceDF['目标订单号']==order_pickup.iloc[z,0])&(DistanceDF['目标类型']==order_pickup.iloc[z,1]),'距离'].values[0]
        except:
            print(z,len(order_pickup))
    
    # 时间
    order_assign['理论时间'] = ((order_assign.loc[:,'距离'] / shuxing.iloc[0,1])+last_action['时间']).apply(int)
    order_pickup['理论时间'] = ((order_pickup.loc[:,'距离'] / shuxing.iloc[0,1])+last_action['时间']).apply(int)

    # 补充订单可取时间，承诺送达时间,天气
    order_assign['订单分配时间'] =0
    order_assign['订单可取时间'] = 0
    order_assign['订单承诺送达时间'] = 0
    order_assign['天气'] = '1'   
    order_pickup['订单分配时间'] =0
    order_pickup['订单可取时间'] = 0
    order_pickup['订单承诺送达时间'] = 0
    order_pickup['天气'] = '1'
    
    for y in range(len(order_assign)):
        oneOrder = OrderDF.loc[OrderDF['订单号'] == order_assign.iloc[y,0],['通知时间','可以取单时间','承诺送达时间','天气']]
        
        order_assign.iloc[y,7] = oneOrder.iloc[0,0]
        order_assign.iloc[y,8] = oneOrder.iloc[0,1]
        order_assign.iloc[y,9] = oneOrder.iloc[0,2]
        order_assign.iloc[y,10] = oneOrder.iloc[0,3]
    
    for z in range(len(order_pickup)):
    
        oneOrder = OrderDF.loc[OrderDF['订单号'] == order_pickup.iloc[z,0],['通知时间','可以取单时间','承诺送达时间','天气']]
       
        order_pickup.iloc[z,7] = oneOrder.iloc[0,0]
        order_pickup.iloc[z,8] = oneOrder.iloc[0,1]
        order_pickup.iloc[z,9] = oneOrder.iloc[0,2]
        order_pickup.iloc[z,10] = oneOrder.iloc[0,3]
    
    order_assign.rename(columns={'订单号':'取单号'},inplace=True)
    order_pickup.rename(columns={'订单号':'送单号'},inplace=True)
   
    return order_assign,order_pickup

In [12]:
testDataList =list()
for n in range(len(test_action_list)):
  
    # 以一天的数据为单位进行处理
    action = test_action_list[n]
    courier = test_courier_list[n]
    distance = test_distance_list[n]
    order = test_order_list[n]
    
    
    
    # 修改列名
    action.rename(columns=action_columns_name_dict,inplace=True)
    courier.rename(columns=courier_columns_name_dict,inplace=True)
    distance.rename(columns=distance_columns_name_dict,inplace =True)
    order.rename(columns=order_columns_name_dict,inplace =True)
    
    
    # 组合订单3个动作的表
    order_part = order[['骑手ID','波次','订单号','通知时间']]
    order_part['动作类型'] ='ASSIGN'
    order_part.rename(columns={'通知时间':'时间'},inplace = True)
    action_part = action[['骑手ID','波次','订单号','动作类型','时间']]
    order_part = order_part[['骑手ID','波次','订单号','动作类型','时间']]
    action_all = pd.concat([action_part,order_part]) # 纵向合并成一个表
    action_all_sort = action_all.sort_values(by =['骑手ID','波次','时间']) # 排序，将骑手的一个波次合并在一起
    
    # 补充动作表对应的地点经纬度
    distance_part = distance[['骑手ID','波次','源订单号','源类型','源经度','源纬度']]
    distance_part.rename(columns={'源订单号':'订单号','源类型':'动作类型','源经度':'经度','源纬度':'纬度'},inplace = True)
    distance_part1 = distance_part.drop_duplicates() # 删除重复行
    action_big = pd.merge(left=action_all_sort,right=distance_part1,left_on=['骑手ID','波次','订单号','动作类型'],right_on=['骑手ID','波次','订单号','动作类型'],how ='inner')
    
    
    dataSort = action[['骑手ID','波次']]
    dataSort.drop_duplicates(inplace =True)  # 留下每个骑手每个波次号
    dataSort['订单号'] = 0
    dataSort['动作'] ='pickup'
    dataSort['起始经度'] =0
    dataSort['起始纬度'] =0

    
     # 生成最后总表初始表
    
    columnsName =['取单号','商家经度','商家纬度','商家距离','理论取单时间','取单可取时间','取单承诺送达时间','送单号','客户经度','客户纬度','客户距离','理论送单时间','送单可取时间','送单承诺送达时间','骑手等级','骑手速度','最大负荷量','当前已取单数']
    for st in columnsName:
        dataSort.loc[:,st] = 0
    dataSort =dataSort.sort_values(by=['骑手ID','波次']) # 让一个骑手的波次聚在一块
    # 修改索引，使其从0递增
    indexlist = list(range(len(dataSort)))
    dataSort.index = indexlist
    
    # 模型读取
    path = '../user_data/model_data/choose_model/'
    
    modelTwoDiff = joblib.load(path+'RFpick2.pkl')
    modelTwoSame = joblib.load(path+'RFpickSame2.pkl')
    
    modelPick3lin = joblib.load(path+'RFpick3.pkl') # 三订单不同店取单模型
    modelPick3PartSamelin = joblib.load(path+'RFpickpartSame3.pkl') # 三订单部分同店取单模型
    modelPick3Samelin = joblib.load(path+'RFpickSame3.pkl') # 三订单都同店取单模型
    
    modelDe2lin = joblib.load(path+'RFDE2.pkl') # 双订单不同店送单模型
    modelDeSame2lin = joblib.load(path+'RFDESame2.pkl') # 双订单同店送单模型
    #modelDe2Feng = joblib.load(path+'RFDE2feng.pkl') # 双订单不同店送单模型(冯)
    
    modelDe3lin = joblib.load(path+'RFDE3.pkl')  # 三订单不同店送单
    modelDePartSame3lin = joblib.load(path+'RFDEpartSame3.pkl') # 三订单部分同店送单
    
    
    #print(dataSort.info())
    for i in range(len(dataSort)):
        rideID = dataSort.iloc[i,0]
        waveID = dataSort.iloc[i,1]
        dataSort_wave = action.loc[(action['骑手ID']==rideID) & (action['波次']==waveID),:]
        finish_wave = dataSort_wave.loc[action['时间'] !=0,:]
        orderID = finish_wave.iloc[-1,2]
        actionID = finish_wave.iloc[-1,5]
        dataSort.loc[i,'订单号'] = orderID
        dataSort.loc[i,'动作'] = actionID
        dataSort.loc[i,'起始经度'] = finish_wave.iloc[-1,3]
        dataSort.loc[i,'起始纬度'] = finish_wave.iloc[-1,4]

        action_wave = action_big.loc[(action_big['骑手ID']==rideID) & (action_big['波次']==waveID),['订单号','动作类型','时间','经度','纬度']] # 包含了分配动作
        # 骑手属性
        shuxing =courier.loc[courier['骑手ID']==rideID,['等级','速度','最大负荷量']]
        # 波次内的距离
        distance_wave = distance.loc[(distance['骑手ID']==rideID) & (distance['波次']==waveID),:]
        # 波次内的订单
        order_wave = order.loc[(order['骑手ID']==rideID) &(order['波次']==waveID),:]
        
        order_assign,order_pickup = OneOrderActionGetAllOption(orderID,actionID,action_wave,order_wave,distance_wave,shuxing)
       
        #取单
        index=0
        if len(order_assign) ==1:
            index = 0
        elif len(order_assign)==2:
            dis = order_assign.iloc[0,5] - order_assign.iloc[1,5]
            time1 = order_assign.iloc[0,6] - order_assign.iloc[1,6] # 理论时间差
            time2 = order_assign.iloc[0,7] - order_assign.iloc[1,7] # 通知时间差
            time3 = order_assign.iloc[0,8] - order_assign.iloc[1,8] # 可取时间差
            time4 = order_assign.iloc[0,9] - order_assign.iloc[1,9] # 承诺时间差
            if dis ==0:
                temp = [[time2,time3,time4]]
                input1 = np.array(temp)
                index = int(modelTwoSame.predict(input1)[0])
            elif (dis>1500) |(dis<-1500):
                order_assign.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
                index=0
            else:
                temp = [[time1,time2,time3,time4]]
                input1 = np.array(temp)
                index = int(modelTwoDiff.predict(input1)[0])
        elif len(order_assign)==3:
            
            dis1 = order_assign.iloc[0,5] - order_assign.iloc[1,5]
            dis2 = order_assign.iloc[0,5] - order_assign.iloc[2,5]
            
            time4 = order_assign.iloc[0,7] - order_assign.iloc[1,7] # 1-2通知时间差
            time5 = order_assign.iloc[0,7] - order_assign.iloc[2,7] # 1-3通知时间差
            time6 = order_assign.iloc[1,7] - order_assign.iloc[2,7] # 2-3通知时间差

            time7 = order_assign.iloc[0,8] - order_assign.iloc[1,8] # 1-2可取时间差
            time8 = order_assign.iloc[0,8] - order_assign.iloc[2,8] # 1-3可取时间差
            time9 = order_assign.iloc[1,8] - order_assign.iloc[2,8] # 2-3可取时间差

            time10 = order_assign.iloc[0,9] - order_assign.iloc[1,9] # 1-2承诺时间差
            time11 = order_assign.iloc[0,9] - order_assign.iloc[2,9] # 1-3承诺时间差
            time12 = order_assign.iloc[1,9] - order_assign.iloc[2,9] # 2-3承诺时间差

            if (abs(dis1)<0.001) &(abs(dis2)<0.001): # 三订单同一家店 
                temp = [[time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index = int(modelPick3Samelin.predict(input1)[0])
            elif (abs(dis1) >0.1) &(abs(dis2)>0.1): # 三订单不同店
                time1 = order_assign.iloc[0,6] - order_assign.iloc[1,6] # 1-2理论时间差
                time2 = order_assign.iloc[0,6] - order_assign.iloc[2,6] # 1-3理论时间差
                time3 = order_assign.iloc[1,6] - order_assign.iloc[2,6] # 2-3理论时间差
                temp = [[time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index = int(modelPick3lin.predict(input1)[0])
            else:
                time1 = order_assign.iloc[0,6] - order_assign.iloc[1,6] # 1-2理论时间差
                time2 = order_assign.iloc[0,6] - order_assign.iloc[2,6] # 1-3理论时间差
                time3 = order_assign.iloc[1,6] - order_assign.iloc[2,6] # 2-3理论时间差
                temp = [[time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index = int(modelPick3PartSamelin.predict(input1)[0])
        else:
            order_assign.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
            index =0
           
        # 送单
        index1 =0
        if len(order_pickup) ==1:
            index1 =0
        elif len(order_pickup) ==2:
            dis = order_pickup.iloc[0,5] - order_pickup.iloc[1,5]
            time1 = order_pickup.iloc[0,6] - order_pickup.iloc[1,6] # 理论时间差
            time2 = order_pickup.iloc[0,7] - order_pickup.iloc[1,7] # 通知时间差
            time3 = order_pickup.iloc[0,8] - order_pickup.iloc[1,8] # 可取时间差
            time4 = order_pickup.iloc[0,9] - order_pickup.iloc[1,9] # 承诺时间差
            if abs(dis)<0.001: # 同单
                temp = [[time2,time4]]
                input1 = np.array(temp)
                index1 = int(modelDeSame2lin.predict(input1)[0])
            elif (dis>1000) |(dis<-1000):
                order_pickup.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
                index1=0
            else:
                #order_pickup.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
                temp = [[time1,time2,time3,time4]]
                input1 = np.array(temp)
                index1 = int(modelDe2lin.predict(input1)[0])
        elif len(order_pickup) ==3:
            dis1 = order_pickup.iloc[0,5] - order_pickup.iloc[1,5]
            dis2 = order_pickup.iloc[0,5] - order_pickup.iloc[2,5]
            
            time1 = order_pickup.iloc[0,6] - order_pickup.iloc[1,6] # 1-2理论时间差
            time2 = order_pickup.iloc[0,6] - order_pickup.iloc[2,6] # 1-3理论时间差
            time3 = order_pickup.iloc[1,6] - order_pickup.iloc[2,6] # 2-3理论时间差
            
            time10 = order_pickup.iloc[0,9] - order_pickup.iloc[1,9] # 1-2承诺时间差
            time11 = order_pickup.iloc[0,9] - order_pickup.iloc[2,9] # 1-3承诺时间差
            time12 = order_pickup.iloc[1,9] - order_pickup.iloc[2,9] # 2-3承诺时间差

            if (abs(dis1)<0.001) &(abs(dis2)<0.001): # 三订单同一家店
                order_pickup.sort_values(by=['订单承诺送达时间'],inplace =True)
                index1 =0
            elif (abs(dis1) >0.1) &(abs(dis2)>0.1): # 三订单不同店
                temp = [[time1,time2,time3,time10,time11,time12]]
                input1 = np.array(temp)
                index1 = int(modelDe3lin.predict(input1)[0])
            else:
                time4 = order_pickup.iloc[0,7] - order_pickup.iloc[1,7] # 1-2通知时间差
                time5 = order_pickup.iloc[0,7] - order_pickup.iloc[2,7] # 1-3通知时间差
                time6 = order_pickup.iloc[1,7] - order_pickup.iloc[2,7] # 2-3通知时间差

                time7 = order_pickup.iloc[0,8] - order_pickup.iloc[1,8] # 1-2可取时间差
                time8 = order_pickup.iloc[0,8] - order_pickup.iloc[2,8] # 1-3可取时间差
                time9 = order_pickup.iloc[1,8] - order_pickup.iloc[2,8] # 2-3可取时间差
                
                temp = [[time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index1 = int(modelDePartSame3lin.predict(input1)[0])
        else:
            order_pickup.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
            index1 =0
        
        
       
        if len(order_assign)==0:
            pass
        else:

            dataSort.loc[i,'取单号'] = order_assign.iloc[index,0]
            dataSort.loc[i,'商家经度'] = order_assign.iloc[index,3]
            dataSort.loc[i,'商家纬度'] = order_assign.iloc[index,4]
            dataSort.loc[i,'商家距离'] = order_assign.iloc[index,5]
            
            dataSort.loc[i,'理论取单时间'] = order_assign.iloc[index,6]
           
            dataSort.loc[i,'取单可取时间'] = order_assign.iloc[index,8]
            dataSort.loc[i,'取单承诺送达时间'] = order_assign.iloc[index,9]
           

        if len(order_pickup) ==0:
            pass
        else:
            dataSort.loc[i,'送单号'] = order_pickup.iloc[index1,0]
            dataSort.loc[i,'客户经度'] = order_pickup.iloc[index1,3]
            dataSort.loc[i,'客户纬度'] = order_pickup.iloc[index1,4]
            dataSort.loc[i,'客户距离'] = order_pickup.iloc[index1,5]
            
            dataSort.loc[i,'理论送单时间'] = order_pickup.iloc[index1,6]
            
            dataSort.loc[i,'送单可取时间'] = order_pickup.iloc[index1,8]
            dataSort.loc[i,'送单承诺送达时间'] = order_pickup.iloc[index1,9]
        
        dataSort.loc[i,'骑手等级'] = shuxing.iloc[0,0]
        dataSort.loc[i,'骑手速度'] = shuxing.iloc[0,1]
        dataSort.loc[i,'最大负荷量'] =shuxing.iloc[0,2]
        dataSort.loc[i,'当前已取单数'] = len(order_pickup)
       
        
    # 调整列名
    data_save=dataSort.reindex(columns=['订单号','动作','起始经度','起始纬度','取单号','商家经度','商家纬度','商家距离','理论取单时间','取单可取时间','取单承诺送达时间','送单号','客户经度','客户纬度','客户距离','理论送单时间','送单可取时间','送单承诺送达时间','骑手ID','波次','当前已取单数','骑手等级','骑手速度','最大负荷量'])
    
   
    data_save['日期'] = data_list[n]
    
    testDataList.append(data_save)

In [13]:
# 保存
for i in range(len(testDataList)):
    testDataList[i].to_csv('../user_data/tmp_data/decision_'+data_list[i]+'.txt',index=False,encoding='utf-8')